In [1]:
from neo4j_utility import *
from llm_extraction import *
from firecrawl_scraping import *
from utility import *
import os
from tqdm import tqdm
from dotenv import load_dotenv

/Users/chenkangan/Documents/UCL DSML/Thesis/project/ucl_project/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Introduction
This notebook will demonstrate the full pipeline of KG consturction from data scraping, information extraction and KG construction.

The starting point of the entire pipeline is the company's URL. In this project, the company's URL can be obtained in two ways:
- From Pitchbook Dataset ('data/merge_url_companies.csv')
- Search company's name based on its name (function: get_and_verify_company_link)

In [10]:
# Method 1: Pitchbook dataset
df = pd.read_csv('data/merge_url_companies.csv')

print(f"Total number of companies: {len(df)}")
print(f"Total number of relevant pages: {df['num_of_related_urls'].sum()}")
print(f"Total number of relevant pages per company: {df['num_of_related_urls'].sum()/len(df)}")

df.head(3)

Total number of companies: 791
Total number of relevant pages: 3217
Total number of relevant pages per company: 4.067003792667509


,company_id,companies,company_former_name,company_legal_name,competitors,description,primary_industry_sector,primary_industry_group,primary_industry_code,all_industries,...,last_known_valuation_deal_type,processed_url,is_accessible,processed_name,url,related_urls_str,related_urls,num_of_related_urls,all_urls,num_of_all_urls
0,55185-04,Estimize,NaN,"Estimize, Inc.","Neudata, SigFig, Motif (Financial Software), Y...",Developer of an open financial estimates platf...,Information Technology,Software,Financial Software,"Financial Software*, Media and Information Ser...",...,Early Stage VC,www.estimize.com,True,estimize,https://www.estimize.com,https://www.estimize.com,['https://www.estimize.com'],1,['https://www.estimize.com'],1.0
1,56288-62,New Constructs,NaN,"New Constructs, LLC","Morningstar, CFRA, Finbox (Media and Informati...",Operator of an investment research firm intend...,Information Technology,Software,Financial Software,"Financial Software*, Media and Information Ser...",...,Early Stage VC,www.newconstructs.com,True,new_constructs,https://www.newconstructs.com,https://www.newconstructs.com/customer-testimo...,['https://www.newconstructs.com/customer-testi...,3,"['https://www.newconstructs.com', 'https://www...",24.0
2,58565-08,RealScout,NaN,"RealScout, Inc.","Spark (Business/Productivity Software), Proper...",The company's platform combines buyer data-dri...,Information Technology,Software,Business/Productivity Software,"Business/Productivity Software*, Media and Inf...",...,Early Stage VC,www.realscout.com,True,realscout,https://www.realscout.com,https://www.realscout.com,['https://www.realscout.com'],1,"['https://www.realscout.com/privacy', 'https:/...",5.0


In [3]:
# Method 2: Company URL retrival 
# Company URL retrival utilise Clearbit's API and Google Custom Search API

get_and_verify_company_link(company_name = 'Post Urban')

Company Post Urban: The primary URL is: https://www.posturban.vc


'https://www.posturban.vc'

## Data Scraping

After getting the company's URL, Firecrawl API can be used to scrape the webpages.

To demonstrate the use case, a sample company 'DAVO Technologies' will be used as an example. A user only need to specify the company name and the URL.

In [2]:
# User input
company_name = 'DAVO Technologies'
url = "https://www.davosalestax.com"


In [3]:
# Define the directory of all relevant files
processed_name = process_company_name(company_name)
scrape_file_dir = f'test/scraping_output_v2_raw'
summary_file_dir = f'test/extraction_summary_v2'
extraction_file_dir = f'test/extraction_output_v2'
client_file_dir = f'test/client_info.json'
scrape_file_path = f'{scrape_file_dir}/{processed_name}.json'
summary_file_path = f'{summary_file_dir}/{processed_name}_summary.json'
extraction_file_path = f'{extraction_file_dir}/{processed_name}_extraction.json'

Only the base URL is not sufficient to collect all relevant data about product offering and clients. 

Use function get_related_urls to fetch all relevant URLs based on keywords.

In [14]:
all_urls, related_urls = get_related_urls(url)
print('Related urls about products and clients:')
related_urls

/Users/chenkangan/Documents/UCL DSML/Thesis/project/ucl_project/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.davosalestax.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Related urls about products and clients:


['https://www.davosalestax.com',
 'https://www.davosalestax.com/partners/',
 'https://www.davosalestax.com/partners/bookkeep/',
 'https://www.davosalestax.com/customer-stories/',
 'https://www.davosalestax.com/partners/back-office/',
 'https://www.davosalestax.com/partner-referral-program/']

In [16]:
# Scrape data from all related URLs

crawl_data(base_url = url, 
           url_list = related_urls,
           file_path = scrape_file_path,
           overwrite = False)

Parent directory already exists: test/scraping_output_v2_raw
Skipping https://www.davosalestax.com as it already exists and overwrite is set to False.
Skipping https://www.davosalestax.com/partners/ as it already exists and overwrite is set to False.
Skipping https://www.davosalestax.com/partners/bookkeep/ as it already exists and overwrite is set to False.
Skipping https://www.davosalestax.com/customer-stories/ as it already exists and overwrite is set to False.
Skipping https://www.davosalestax.com/partners/back-office/ as it already exists and overwrite is set to False.
Skipping https://www.davosalestax.com/partner-referral-program/ as it already exists and overwrite is set to False.


{'processed_company': 'davo_technologies',
 'url': 'https://www.davosalestax.com',
 'timestamp': '2024-08-18 15:44 Etc/GMT',
 'main_page': 'Automate your sales tax.\n========================\n\n*   Sets aside sales tax daily\n*   Files and pays, on time & in full\n*   Guaranteed\n\n[See Our Guarantee Policy](https://legal.avalara.com/product-terms.html#davo)\n\n[Try DAVO for free](https://www.davosalestax.com/integrations/)\n\nSeamlessly integrates with the most popular POS systems.\n--------------------------------------------------------\n\n### Click yours to get started.\n\n[![Clover logo](https://www.davosalestax.com/wp-content/uploads/2024/07/Clover-logo-horizontal-CloverGreen-400.png)](https://www.davosalestax.com/integrations/clover/)\n\n[![Square logo](https://www.davosalestax.com/wp-content/uploads/2022/05/square-new.png)](https://www.davosalestax.com/integrations/square/)\n\n[![Toast Logo](https://www.davosalestax.com/wp-content/uploads/2019/12/toast-logo.png)](https://www.da

## Information Extraction Prompting Chain

A three-layer prompting chain was implemented to manage this process. 

- The first layer, content filtering, identified and filtered relevant information from the scraped content. 
- The second layer, semantic data extraction, focused on extracting and structuring product and client information. 
- The final layer, client validation and product matching, classified client entities and searched for their URLs, while establishing links between clients and products.

In [4]:

# Layer 1
_ = llm_summary_execution(processed_name = processed_name,
                        scrape_file_path = scrape_file_path,
                        summary_file_path = summary_file_path,
                        overwrite = True,
                        model_name = 'gpt-4o-mini')

# Layer 2 & 3
_ = llm_extraction_execution(processed_name = processed_name,
                        summary_file_path = summary_file_path,
                        extraction_file_path = extraction_file_path, 
                        include_additional_context = True, 
                        overwrite = True,
                        model_name = 'gpt-4o')

# Get the embedding of product description
_ = get_product_embedding(processed_name = processed_name,
                    extraction_file_path = extraction_file_path,
                    embedding_model = "text-embedding-3-small")

# Add the client's URLs to the extraction output
_ = add_client_url_to_extraction_output(processed_name = processed_name,
                            extraction_file_path = extraction_file_path)

# Trouble shoot any issues
_ = troubleshoot_llm_output(processed_name = processed_name,
                    extraction_file_path = extraction_file_path)

# Update the client list (All clients' URLs are stored in data/client_info.json)
_ = update_client_list(processed_name = processed_name,
                extraction_file_path = extraction_file_path,
                client_file_path = client_file_dir)

Parent directory already exists: test/extraction_summary_v2
Company: davo_technologies; Content in main_page is extracted.
Company: davo_technologies; Content in /partners/back-office/ is extracted.
Company: davo_technologies; Content in /partners/bookkeep/ is extracted.
Company: davo_technologies; Content in /partner-referral-program/ is extracted.
Company: davo_technologies; Content in /partners/ is extracted.
Company: davo_technologies; Content in /customer-stories/ is extracted.
Parent directory already exists: test/extraction_output_v2
Company: davo_technologies; Information extraction begins.
Company: davo_technologies; Estimated Cost: $0.009245
Company: davo_technologies; Pitchbook description obtained: Developer of an automated sales tax remedy platform designed to assist organizations with daily cash management. The company's platform automatically collects, files and pays sales tax for a merchant utilizing the ACH and card payments networks and integrates with a client's poin

## Knowledge Graph Construction

Knowledge graph will then be constructed based on the extraction file.

In [5]:
kg_construction(processed_name = processed_name, 
                extraction_file_path = extraction_file_path)

Company davo_technologies is added to the graph.


The graph can be seen in Neo4j's browser.

<div>
<img src="sample_mini_graph.png" width="500"/>
</div>

## Knowledge Graph Construction (Scaling to All Companies)
In the previous sections, a sample company goes through the data scraping, information extraction and the KG construction process.

In this project, all the data scraping and information extraction have been completed.

This section will take all the existing extraction files and add them to the KG.

In [8]:
# Load all the FinTech companies to the KG

fintech_extraction_path = 'extraction_output_v2'

doc_list = os.listdir(fintech_extraction_path)

# Only add 5 companies as a demo
for doc in doc_list[:5]:
    try:
        processed_name = doc.replace('_extraction.json', '')
        extraction_file_path = f'{fintech_extraction_path}/{doc}'
        kg_construction(processed_name, extraction_file_path)
    except Exception as e:
        print(f'Error occurs on company {processed_name}, {e}')

Company hearro is added to the graph.
Company raken is added to the graph.
Company crunch_ is added to the graph.
Company crowdbureau is added to the graph.
Company yayem is added to the graph.


In [9]:
# Load all the client companies to the KG

client_extraction_path = 'client_extraction_output'

doc_list = os.listdir(client_extraction_path)

# Only add 5 companies as a demo
for doc in doc_list[:5]:
    try:
        processed_name = doc.replace('_extraction.json', '')
        extraction_file_path = f'{client_extraction_path}/{doc}'
        kg_construction(processed_name, extraction_file_path)
    except Exception as e:
        print(f'Error occurs on company {processed_name}, {e}')

Company northmarq is added to the graph.
Company np_dodge is added to the graph.
Company unilever is added to the graph.
Company coca_cola is added to the graph.
Company conner_strong_&_buckelew is added to the graph.


In [11]:
doc_list = os.listdir('client_extraction_summary')
num_companies = 0
num_pages = 0
for doc in doc_list:
    try:
        data = read_json_file(f'client_extraction_summary/{doc}')
        pages = data.keys()
        num_companies += 1
        valid_pages = [i for i in pages if i not in ['processed_name', 'timestamp', 'url', 'model_name']]
        num_pages += len(valid_pages)
    except Exception as e:
        print(f'Errors found at {doc}: {e}')
        
print(f"Total number of companies: {num_companies}")
print(f"Total number of relevant pages: {num_pages}")
print(f"Total number of relevant pages per company: {num_pages/num_companies}")

Errors found at .DS_Store: 'utf-8' codec can't decode byte 0x80 in position 3131: invalid start byte
Total number of companies: 1413
Total number of relevant pages: 7545
Total number of relevant pages per company: 5.339702760084926


### Trouble shooting

In [ ]:
def troubleshoot_llm_output(processed_name:str, extraction_file_path:str):
    
    try:
        data = read_json_file(extraction_file_path)
        clients = data['validated_client_descriptions']
        modified = False
        product_list = [product['name'] for product in data['product_descriptions']] + [data['summary_product_description']['name']]
        
        if clients:
            for client in clients:
                # Troubleshoot 1: If the client's URL == company's URL
                if client['url']:
                    if client['url'] == data["url"]:
                        print(f'Issue found: Client URL == Company URL: Company {processed_name}; Client {client["name"]}; {client["url"]}')
                        client['url'] = None
                        modified = True
                        
                # Troubleshoot 2: If product_used is not in the product list previously extracted
                if client['product_used'] not in product_list:
                    print(f'Issue found: Company {processed_name}; Client {client["name"]}; Product used {client["product_used"]}')
                    client['product_used'] = data['summary_product_description']['name']
                    modified = True
                    
        if modified:
            write_json_file(extraction_file_path, data)
    except Exception as e:
        print(f'Error found at {processed_name}: {e}')
            
    

### Troubleshooting 1
Issues encountered when client's URL = company's URL

Solution: Set those URLs to None

In [5]:
doc_list = os.listdir('extraction_output_v2_original')
for doc in doc_list:
    
    try:
        processed_name = doc.replace('_extraction.json', '')
        modified = False
        extraction_file_path = f'extraction_output_v2_original/{processed_name}_extraction.json'
        data = read_json_file(extraction_file_path)
        
        clients = data['validated_client_descriptions']
        
        if clients:
            for client in clients:
                if client['url']:
                    if client['url'] == data["url"]:
                        print(f'Issue found: Company {processed_name}; Client {client["name"]}; {client["url"]}')
        #                 client['url'] = None
        #                 modified = True
                        
        # if modified:
        #     write_json_file(extraction_file_path, data)
    except Exception as e:
        print(f'Error found at {processed_name}: {e}')
    

Issue found: Company goodfolio; Client GOODFOLIO; https://www.goodfolio.com
Issue found: Company client_hub; Client Logic Accounting Solutions, Inc; https://www.clienthub.app
Error found at .DS_Store: [Errno 2] No such file or directory: 'extraction_output_v2_original/.DS_Store_extraction.json'
Issue found: Company hemlane; Client HP2 RESIDENTIAL; https://www.hemlane.com
Issue found: Company per_diem; Client Masala Wok and Tikka Shack; https://www.tryperdiem.com
Issue found: Company nami_ml; Client Toronto App Factory; https://www.namiml.com
Issue found: Company likely_ai; Client 1000calls; https://www.likely.ai
Issue found: Company homesearch; Client Peaksons Property Limited; https://www.homesearch.co.uk
Issue found: Company workmarket; Client Quantum Installations Group; https://www.workmarket.com
Error found at celebrity_agent: 'url'
Issue found: Company pf_nexus; Client Debt Capital Advisor Firm; https://www.pfnexus.com
Issue found: Company pf_nexus; Client Basso Group Capital Adv

### Troubleshooting 2
Issues encountered when product used is not in the product list (including NaN)

Solution: use the summary product node

In [6]:
doc_list = os.listdir('extraction_output_v2_original')
for doc in doc_list:
    
    try:
        processed_name = doc.replace('_extraction.json', '')
        modified = False
        extraction_file_path = f'extraction_output_v2_original/{processed_name}_extraction.json'
        data = read_json_file(extraction_file_path)
        
        product_list = [product['name'] for product in data['product_descriptions']] + [data['summary_product_description']['name']]
        
        clients = data['validated_client_descriptions']
        
        if clients:
            for client in clients:
                if client['product_used'] not in product_list:
                    print(f'Issue found: Company {processed_name}; Client {client["name"]}; Product used {client["product_used"]}; URL: {data["url"]}')
        #             client['product_used'] = data['summary_product_description']['name']
        #             modified = True
        
        # if modified:
        #     write_json_file(extraction_file_path, data)
                        
    except Exception as e:
        print(f'Error found at {processed_name}: {e}')

Issue found: Company raken; Client Level 10; Product used None; URL: https://www.rakenapp.com
Issue found: Company raken; Client Webcor; Product used None; URL: https://www.rakenapp.com
Issue found: Company raken; Client Barton Malow; Product used None; URL: https://www.rakenapp.com
Issue found: Company raken; Client Danforth; Product used None; URL: https://www.rakenapp.com
Issue found: Company raken; Client Wadman Corp; Product used None; URL: https://www.rakenapp.com
Issue found: Company raken; Client Fortis Construction; Product used None; URL: https://www.rakenapp.com
Issue found: Company raken; Client D.L. Henricksen; Product used None; URL: https://www.rakenapp.com
Issue found: Company raken; Client Central Plumbing; Product used None; URL: https://www.rakenapp.com
Issue found: Company raken; Client Kyne Construction; Product used None; URL: https://www.rakenapp.com
Issue found: Company crowdbureau; Client CoreLogic; Product used None; URL: https://www.crowdbureau.com
Issue foun